In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from allensdk.core.cell_types_cache import CellTypesCache

In [ ]:
!mkdir -p cell_types

In [ ]:
# Instantiate the CellTypesCache instance.  The manifest_file argument
# tells it where to store the manifest, which is a JSON file that tracks
# file paths.  If you supply a relative path (like this), it will go
# into your current working directory
ctc = CellTypesCache(manifest_file='./cell_types_manifest.json')

# this saves the NWB file to 'cell_types/specimen_464212183/ephys.nwb'
data_set = ctc.get_ephys_data(464212183)

sweep_number = 33
sweep_data = data_set.get_sweep(sweep_number)

index_range = sweep_data["index_range"]
i = sweep_data["stimulus"][0:index_range[1]+1] # in A
v = sweep_data["response"][0:index_range[1]+1] # in V
i *= 1e12 # to pA
v *= 1e3 # to mV

sampling_rate = sweep_data["sampling_rate"] # in Hz

In [ ]:
## sub-sampling
#v = v[0::200]
#i = i[0::200]
#t = np.arange(0, len(v)) * (200000.0 / sampling_rate)

t = np.arange(0, len(v)) * (1000 / sampling_rate)

###############################################################################
# detect spikes
v1 = np.array(v)
# put everything to -10 that is below -10 or has negative slope
ind = np.where(v1 < -10)
v1[ind] = -10
ind = np.where(np.diff(v1) < 0)
v1[ind] = -10

# remaining negative slopes are at spike peaks
ind = np.where(np.diff(v1) < 0)
spike_times = np.array(t)[ind]
spike_times_stim = spike_times[(spike_times > 1020) & (spike_times < 2020)]
print(spike_times_stim.shape)

# choose one spike time within close spike times
spike_times_stim = spike_times_stim[np.append(1,np.diff(spike_times_stim))>0.5]
print(spike_times_stim.shape)

###############################################################################

#plt.style.use('ggplot')
plt.style.use('classic')
fig, axes = plt.subplots(2, 1, sharex=True)
axes[0].plot(t, v, color='black')
axes[1].plot(t, i, color='gray')
#axes[0].plot(t[int(1.01*sampling_rate):int(2.1*sampling_rate)], v[int(1.01*sampling_rate):int(2.1*sampling_rate)], color='black')
#axes[0].plot(spike_times_stim, 22+np.zeros_like(spike_times_stim), '.r')
#axes[1].plot(t[int(1.01*sampling_rate):int(2.1*sampling_rate)], i[int(1.01*sampling_rate):int(2.1*sampling_rate)], color='gray')
axes[0].set_ylabel("mV")
axes[1].set_ylabel("pA")
axes[1].set_xlabel("milliseconds")
plt.show()